# Agentic RAG with Multi-document + Tools Calling + ReAct

## Preparation Stage

In [1]:
%pip install \
"python-dotenv==1.0.0" \
"openai==1.23.2" \
"llama-index==0.10.27" \
"llama-index-core==0.10.27" \
"llama-index-llms-openai==0.1.15" \
"llama-index-embeddings-openai==0.1.7" \
"llama-index-agent-openai==0.2.2" \
"nest-asyncio==1.6.0" \
"httpx==0.27.2"

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip list

Package                                 Version
--------------------------------------- ------------
aiohappyeyeballs                        2.4.4
aiohttp                                 3.11.10
aiosignal                               1.3.2
annotated-types                         0.7.0
anyio                                   4.7.0
asttokens                               3.0.0
attrs                                   24.3.0
azure-ai-vision-imageanalysis           1.0.0b3
azure-core                              1.32.0
beautifulsoup4                          4.12.3
certifi                                 2024.12.14
charset-normalizer                      3.4.0
click                                   8.1.7
colorama                                0.4.6
comm                                    0.2.2
contourpy                               1.3.1
cycler                                  0.12.1
dataclasses-json                        0.6.7
debugpy                                 1.8.11
decorator  


[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%%writefile ./.env
OPENAI_API_KEY=sk-GUktlQ4YGz8DdZ1D5f02A83836444b4dAdC7D5F9BcFeFf6d
# replace https://api.openai.com/v1 with API transit address
OPENAI_BASE_URL=https://pro.aiskt.com/v1

Overwriting ./.env


In [4]:
import os
from dotenv import load_dotenv, find_dotenv

def get_openai_api_key():
    _ = load_dotenv(find_dotenv())
    return os.getenv("OPENAI_API_KEY")

def get_openai_base_url():
    _ = load_dotenv(find_dotenv())
    return os.getenv("OPENAI_BASE_URL")

# assign corresponding value to api_key before invoking OpenAI(), once setup here, all the following calling from
# other frameworks like LlamaIndex and Trulens will inherit and don't need to config for the same
import openai
openai.api_key = get_openai_api_key()
openai.base_url = get_openai_base_url()
#OPENAI_API_KEY = get_openai_api_key()
#OPENAI_BASE_URL = get_openai_base_url()

import nest_asyncio
nest_asyncio.apply()

import warnings
warnings.filterwarnings('ignore')

## Router Query Engines of Summary Index and Vector Index

In [5]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import SummaryIndex, VectorStoreIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

def get_router_query_engine(
    file_path: str,
    llm = None,
    embed_model = None,
):
    llm = llm or OpenAI(model="gpt-3.5-turbo")
    embed_model = embed_model or OpenAIEmbedding(model="text-embedding-ada-002")

    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()

    splitter = SentenceSplitter(chunk_size=1024)
    nodes = splitter.get_nodes_from_documents(documents)

    # try to avoid the following issue: 'NoneType' object is not iterable
    '''
    /usr/local/lib/python3.10/dist-packages/openai/resources/embeddings.py in parser(obj)
    --> 101 for embedding in obj.data:
        102     data = cast(object, embedding.embedding)
    TypeError: 'NoneType' object is not iterable
    '''

    summary_index = SummaryIndex(
        nodes,
        #embed_model=embed_model,
    )
    vector_index = VectorStoreIndex(
        nodes,
        #embed_model=embed_model,
    )

    summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize",
        use_async=True,
        llm=llm,
    )
    vector_query_engine = vector_index.as_query_engine(
        llm=llm,
    )

    summary_tool = QueryEngineTool.from_defaults(
        query_engine=summary_query_engine,
        description=(
            "Useful for summarization questions related to MetaGPT"
        ),
    )
    vector_tool = QueryEngineTool.from_defaults(
        query_engine=vector_query_engine,
        description=(
            "Useful for retrieving specific context or particular content in detail from the paper of MetaGPT"
        ),
    )

    query_engine = RouterQueryEngine(
        selector=LLMSingleSelector.from_defaults(),
        query_engine_tools=[
            summary_tool,
            vector_tool,
        ],
        verbose=True,
    )
    return query_engine

In [6]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

In [7]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter

documents = SimpleDirectoryReader(
    input_files=["Building Agentic RAG with Llamaindex/Paper-METAGPT_META_PROGRAMMING_FOR_A_MULTI-AGENT_COLLABORATIVE_FRAMEWORK.pdf"],
).load_data()

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [8]:
from llama_index.core import SummaryIndex, VectorStoreIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context or particular content in detail from the paper of MetaGPT"
    ),
)

In [9]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True,
)

response = query_engine.query(
    "What is the summary of the document?"
)
print(str(response))
print(len(response.source_nodes))

response = query_engine.query(
    "How do agents share information in detail with other agents?"
)
print(str(response))
print(len(response.source_nodes))

Selecting query engine 0: This choice indicates that the document is useful for summarization questions related to MetaGPT, which aligns with the question asking for the summary of the document..
The document introduces MetaGPT, a meta-programming framework for multi-agent collaboration in software development. It assigns specific roles to agents like Product Manager, Engineer, Architect, and Project Manager, breaking down complex tasks into subtasks for streamlined workflows. MetaGPT incorporates Standard Operating Procedures (SOPs) to enhance collaboration efficiency and reduce errors, achieving state-of-the-art performance in code generation benchmarks. The framework emphasizes structured communication, workflow adherence, and iterative programming with executable feedback, showcasing superior performance in handling complex software engineering tasks efficiently. Additionally, MetaGPT leverages role specialization, workflow management, and efficient sharing mechanisms to improve pr

In [10]:
query_engine = get_router_query_engine("Building Agentic RAG with Llamaindex/Paper-METAGPT_META_PROGRAMMING_FOR_A_MULTI-AGENT_COLLABORATIVE_FRAMEWORK.pdf")

response = query_engine.query(
    "Tell me about the ablation study results?"
)
print(str(response))

Selecting query engine 1: Ablation study results typically involve specific details and context from the paper, making choice 2 the most relevant for retrieving this information..
The ablation study results show the performance of MetaGPT without feedback compared to other models like AlphaCode, Incoder, CodeGeeX, GPT-4, PaLM Coder, Codex, and Codex + CodeT in terms of Pass rates on the MBPP and HumanEval datasets.


## Building a Multi-document Agent

In [11]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")

In [13]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, SummaryIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.vector_stores import MetadataFilters, FilterCondition
from llama_index.core.llms import ChatMessage
from typing import List, Optional

def get_doc_tools(
    file_path: str,
    name: str,
) -> str:
    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()

    splitter = SentenceSplitter(chunk_size=1024)
    nodes = splitter.get_nodes_from_documents(documents)

    vector_index = VectorStoreIndex(nodes)

    # vector_query is used to answer questions over a given paper, always leave page_numbers as None unless there is
    # a specific page you want to search for
    def vector_query(
        query: str,
        page_numbers: Optional[List[str]] = None,
    ) -> str:
        page_numbers = page_numbers or []
        metadata_dicts = [
            {"key": "page_label", "value": p} for p in page_numbers
        ]

        query_engine = vector_index.as_query_engine(
            similarity_top_k=2,
            filters=MetadataFilters.from_dicts(
                metadata_dicts,
                condition=FilterCondition.OR,
            ),
        )
        response = query_engine.query(query)

        return response

    # the tool name is expected a string with maximum length 64 of only alphabet and underscore, etc., here fetching
    # the keywords from the long file name
    messages = [
        ChatMessage(
            role="system",
            content="You're good at summarization. The user prompt is a paper name. Offer a best short version \
            according to the below rules: \
            1. Keep the first 3 words exactly same as the original paper name and remove the word paper \
            2. Reserve the keywords of the paper name especially describing the core concept \
            3. Only alphabet and underscore are allowed, truncate the tail if it exceeds 30 characters",
        ),
        ChatMessage(role="user", content=name),
    ]
    # assure naming conventions of the tool name by applying solid programming codes afterwards
    short_name = str(llm.chat(messages).message.content).replace(" ", "_")[:30]
    print("Tool name: " + short_name)

    # FuntionTool allow users to easily convert a user-defined function into a tool, and when using an agent with
    # function calling, the tool selected rely strongly on the name and description of the tools, therefore carefully
    # tuning these parameters can result in larges changes in how LLM calls the tools
    # ref: https://docs.llamaindex.ai/en/stable/module_guides/deploying/agents/tools/
    vector_query_tool = FunctionTool.from_defaults(
        name=f"vector_tool_{short_name}",
        fn=vector_query,
        description=(
            f"Useful for retrieving specific context or particular content in detail from the paper of {name}"
        ),
    )

    summary_index = SummaryIndex(nodes)
    summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize",
        use_async=True,
    )
    summary_tool = QueryEngineTool.from_defaults(
        name=f"summary_tool_{short_name}",
        query_engine=summary_query_engine,
        description=(
            f"Useful for summarization questions related to {name}"
        ),
    )

    return vector_query_tool, summary_tool

In [15]:
urls = [
    "https://openreview.net/pdf?id=VtmBAGCN7o",
    "https://openreview.net/pdf?id=6PmJoRfdaK",
    "https://openreview.net/pdf?id=hSyW5go0v8",
]

papers = [
    "Building Agentic RAG with Llamaindex/Paper-METAGPT_META_PROGRAMMING_FOR_A_MULTI-AGENT_COLLABORATIVE_FRAMEWORK.pdf",
    "Building Agentic RAG with Llamaindex/Paper-LONGLORA_EFFICIENT_FINE-TUNING_OF_LONG-CONTEXT_LARGE_LANGUAGE_MODELS.pdf",
    "Building Agentic RAG with Llamaindex/Paper-SELF-RAG_LEARNING_TO_RETRIEVE_GENERATE_AND_CRITIQUE_THROUGH_SELF-REFLECTION.pdf",
]

In [16]:
from pathlib import Path

paper_to_tools_dict = {}
for paper in papers:
    print(f"Getting tools for paper: {paper}")

    vector_tool, summary_tool = get_doc_tools(paper, Path(paper).stem)
    paper_to_tools_dict[paper] = [vector_tool, summary_tool]

initial_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]
len(initial_tools)

Getting tools for paper: Building Agentic RAG with Llamaindex/Paper-METAGPT_META_PROGRAMMING_FOR_A_MULTI-AGENT_COLLABORATIVE_FRAMEWORK.pdf
Tool name: METAGPT_META_PROGRAMMING_MULTI
Getting tools for paper: Building Agentic RAG with Llamaindex/Paper-LONGLORA_EFFICIENT_FINE-TUNING_OF_LONG-CONTEXT_LARGE_LANGUAGE_MODELS.pdf
Tool name: LONGLO_EFFICIENT_FINE_TUNING
Getting tools for paper: Building Agentic RAG with Llamaindex/Paper-SELF-RAG_LEARNING_TO_RETRIEVE_GENERATE_AND_CRITIQUE_THROUGH_SELF-REFLECTION.pdf
Tool name: SELF-RAG_LEARNING_RETRIEVE_GEN


6

In [17]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    initial_tools,
    llm=llm,
    verbose=True,
)
agent = AgentRunner(agent_worker)

In [18]:
response = agent.query(
    "Tell me about the evaluation dataset used in LongLoRA, and then tell me about the evaluation results"
)
print(str(response), "\n")
response = agent.query(
    "Give me a summary of both Self-RAG and LongLoRA"
)
print(str(response))

Added user message to memory: Tell me about the evaluation dataset used in LongLoRA, and then tell me about the evaluation results
=== Calling Function ===
Calling function: vector_tool_LONGLO_EFFICIENT_FINE_TUNING with args: {"query": "evaluation dataset"}
=== Calling Function ===
Calling function: vector_tool_LONGLO_EFFICIENT_FINE_TUNING with args: {"query": "evaluation results"}
assistant: The evaluation dataset used in LongLoRA is called SAFECONV, which aims to enhance research on conversational safety. It provides annotations for unsafe spans in utterances and suggests safe alternative responses. The dataset includes unsafespans, unsafe responses, and safe alternative responses for over 100,000 dialogues from social media platforms. It is compared with other datasets to demonstrate its comprehensiveness in identifying unsafe spans and rewriting unsafe responses to mitigate unsafe behavior in chatbots.

Regarding the evaluation results, the models in LongLoRA achieved better perple

In [19]:
urls = [
    "https://openreview.net/pdf?id=VtmBAGCN7o",
    "https://openreview.net/pdf?id=6PmJoRfdaK",
    "https://openreview.net/pdf?id=LzPWWPAdY4",
    "https://openreview.net/pdf?id=VTF8yNQM66",
    "https://openreview.net/pdf?id=hSyW5go0v8",
    "https://openreview.net/pdf?id=9WD9KwssyT",
    "https://openreview.net/pdf?id=yV6fD7LYkF",
    "https://openreview.net/pdf?id=hnrB5YHoYu",
    "https://openreview.net/pdf?id=WbWtOYIzIK",
    "https://openreview.net/pdf?id=c5pwL0Soay",
    "https://openreview.net/pdf?id=TpD2aG1h0D",
]

papers = [
    "Building Agentic RAG with Llamaindex/Paper-METAGPT_META_PROGRAMMING_FOR_A_MULTI-AGENT_COLLABORATIVE_FRAMEWORK.pdf",
    "Building Agentic RAG with Llamaindex/Paper-LONGLORA_EFFICIENT_FINE-TUNING_OF_LONG-CONTEXT_LARGE_LANGUAGE_MODELS.pdf",
    "Building Agentic RAG with Llamaindex/Paper-LOFTQ_LORA-FINE-TUNING-AWARE_QUANTIZATION_FOR_LARGE_LANGUAGE_MODELS.pdf",
    "Building Agentic RAG with Llamaindex/Paper-SWE-BENCH_CAN_LANGUAGE_MODELS_RESOLVE_REAL-WORLD_GITHUB_ISSUES.pdf",
    "Building Agentic RAG with Llamaindex/Paper-SELF-RAG_LEARNING_TO_RETRIEVE_GENERATE_AND_CRITIQUE_THROUGH_SELF-REFLECTION.pdf",
    "Building Agentic RAG with Llamaindex/Paper-ZIPFORMER_A_FASTER_AND_BETTER_ENCODER_FOR_AUTOMATIC_SPEECH_RECOGNITION.pdf",
    "Building Agentic RAG with Llamaindex/Paper-VALUES_A_FRAMEWORK_FOR_SYSTEMATIC_VALIDATION_OF_UNCERTAINTY_ESTIMATION_IN_SEMANTIC_SEGMENTATION.pdf",
    "Building Agentic RAG with Llamaindex/Paper-FINETUNING_TEXT-TO-IMAGE_DIFFUSION_MODELS_FOR_FAIRNESS.pdf",
    "Building Agentic RAG with Llamaindex/Paper-KNOWLEDGE_CARD_FILLING_LLMS_KNOWLEDGE_GAPS_WITH_PLUG-IN_SPECIALIZED_LANGUAGE_MODELS.pdf",
    "Building Agentic RAG with Llamaindex/Paper-METRA_SCALABLE_UNSUPERVISED_RL_WITH_METRIC-AWARE_ABSTRACTION.pdf",
    "Building Agentic RAG with Llamaindex/Paper-META_CONTINUAL_LEARNING_REVISITED_IMPLICITLY_ENHANCING_ONLINE_HESSIAN_APPROXIMATION_VIA_VARIANCE_REDUCTION.pdf",
]

In [20]:
from pathlib import Path

paper_to_tools_dict = {}
for paper in papers:
    print(f"Getting tools for paper: {paper}")

    vector_tool, summary_tool = get_doc_tools(paper, Path(paper).stem)
    paper_to_tools_dict[paper] = [vector_tool, summary_tool]

all_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]
len(all_tools)

Getting tools for paper: Building Agentic RAG with Llamaindex/Paper-METAGPT_META_PROGRAMMING_FOR_A_MULTI-AGENT_COLLABORATIVE_FRAMEWORK.pdf
Tool name: METAGPT_Meta_Programming_Multi
Getting tools for paper: Building Agentic RAG with Llamaindex/Paper-LONGLORA_EFFICIENT_FINE-TUNING_OF_LONG-CONTEXT_LARGE_LANGUAGE_MODELS.pdf
Tool name: LONGLO_EFFICIENT_FINE_TUNING
Getting tools for paper: Building Agentic RAG with Llamaindex/Paper-LOFTQ_LORA-FINE-TUNING-AWARE_QUANTIZATION_FOR_LARGE_LANGUAGE_MODELS.pdf
Tool name: LOFTQ_LORA_FINE_TUNING_AWARE_Q
Getting tools for paper: Building Agentic RAG with Llamaindex/Paper-SWE-BENCH_CAN_LANGUAGE_MODELS_RESOLVE_REAL-WORLD_GITHUB_ISSUES.pdf
Tool name: SWE_BENCH_CAN_LANGUAGE_MODELS_
Getting tools for paper: Building Agentic RAG with Llamaindex/Paper-SELF-RAG_LEARNING_TO_RETRIEVE_GENERATE_AND_CRITIQUE_THROUGH_SELF-REFLECTION.pdf
Tool name: SELF_RAG_LEARNING_RETRIEVE_GEN
Getting tools for paper: Building Agentic RAG with Llamaindex/Paper-ZIPFORMER_A_FASTER_AN

22

In [21]:
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

obj_retriever = obj_index.as_retriever(similarity_top_k=3)
tools = obj_retriever.retrieve(
    "Tell me about the evaluation dataset used in MetaGPT and compare it against SWE-Bench, just the brief info is enough"
)
for tool in tools:
  print(tool.metadata)

ToolMetadata(description='Useful for summarization questions related to Paper-SWE-BENCH_CAN_LANGUAGE_MODELS_RESOLVE_REAL-WORLD_GITHUB_ISSUES', name='summary_tool_SWE_BENCH_CAN_LANGUAGE_MODELS_', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>)
ToolMetadata(description='Useful for summarization questions related to Paper-METAGPT_META_PROGRAMMING_FOR_A_MULTI-AGENT_COLLABORATIVE_FRAMEWORK', name='summary_tool_METAGPT_Meta_Programming_Multi', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>)
ToolMetadata(description='Useful for retrieving specific context or particular content in detail from the paper of Paper-SWE-BENCH_CAN_LANGUAGE_MODELS_RESOLVE_REAL-WORLD_GITHUB_ISSUES', name='vector_tool_SWE_BENCH_CAN_LANGUAGE_MODELS_', fn_schema=<class 'pydantic.v1.main.vector_tool_SWE_BENCH_CAN_LANGUAGE_MODELS_'>)


In [22]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_retriever,
    llm=llm,
    system_prompt="You are an agent designed to answer queries over a set of given papers. \
    Please always use the tools provided to answer a question. Do not rely on prior knowledge.",
    verbose=True,
)
agent = AgentRunner(agent_worker)

In [24]:
response = agent.query(
    "Tell me about the evaluation dataset used in MetaGPT and compare it against SWE-Bench, just the brief info is enough"
)
print('--------------------------------------------------------------------------------------------------')
print(str(response), "\n")
response = agent.query(
    "Compare and contrast the LoRA papers (LongLoRA, LoftQ), remember to analyze the detail approach in each paper first"
)
print('--------------------------------------------------------------------------------------------------')
print(str(response))

Added user message to memory: Tell me about the evaluation dataset used in MetaGPT and compare it against SWE-Bench, just the brief info is enough
=== Calling Function ===
Calling function: summary_tool_METAGPT_Meta_Programming_Multi with args: {"input": "evaluation dataset"}
=== Calling Function ===
Calling function: summary_tool_SWE_BENCH_CAN_LANGUAGE_MODELS_ with args: {"input": "evaluation dataset"}
--------------------------------------------------------------------------------------------------
assistant: The evaluation dataset used in MetaGPT consists of HumanEval and MBPP, which include software development tasks with specific requirements and expected outcomes for assessing model performance in code generation. It provides insights into code statistics, document statistics, cost statistics, revision costs, code executability, and encountered errors during coding.

On the other hand, the SWE-Bench dataset is a challenging dataset that includes task instructions, issue text, ret

## CoT and Agent Reasoning Loop

In [25]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

# ref: https://docs.llamaindex.ai/en/stable/examples/agent/react_agent_with_query_engine/
march_2022 = SimpleDirectoryReader(
    input_files=["Building Agentic RAG with Llamaindex/uber_financial_march_2022.pdf"]
).load_data()
june_2022 = SimpleDirectoryReader(
    input_files=["Building Agentic RAG with Llamaindex/uber_financial_june_2022.pdf"]
).load_data()
september_2022 = SimpleDirectoryReader(
    input_files=["Building Agentic RAG with Llamaindex/uber_financial_september_2022.pdf"]
).load_data()

In [27]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")

In [28]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

def get_tool(
    name,
    full_name,
    documents = None,
):
    if not os.path.exists(f"./{name}"):
        vector_index = VectorStoreIndex.from_documents(documents)
        vector_index.storage_context.persist(persist_dir=f"./{name}")
    else:
        vector_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=f"./{name}"),
        )

    query_engine = vector_index.as_query_engine(
        similarity_top_k=3,
        llm=llm,
    )
    query_engine_tool = QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name=name,
            description=(
                "Provides information about Uber quarterly financials ending"
                f" {full_name}"
            ),
        ),
    )

    return query_engine_tool

In [29]:
march_tool = get_tool("march_2022", "March 2022", documents=march_2022)
june_tool = get_tool("june_2022", "June 2022", documents=june_2022)
september_tool = get_tool("september_2022", "September 2022", documents=september_2022)

query_engine_tools = [march_tool, june_tool, september_tool]

In [30]:
from llama_index.core.agent import AgentRunner, ReActAgent
from llama_index.agent.openai import OpenAIAgentWorker, OpenAIAgent
from llama_index.agent.openai import OpenAIAgentWorker

#agent_llm = OpenAI(model="gpt-4o")
agent_llm = OpenAI(model="gpt-3.5-turbo")

agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=agent_llm,
    verbose=True,
    max_iterations=15,
)

In [31]:
'''
response = agent.chat(
    "Analyze the changes in Uber R&D expenditures and revenue in year 2022 and provide a comprehensive report."
)
'''
# leverage the power of CoT and ReAct to make GPT 3.5 think like 4
response = agent.chat(
    "Analyze the changes in Uber R&D expenditures and revenue in year 2022 and provide a comprehensive report, you need to \
    gather information from the quarterly financials ending Mar. 2022, Jun. 2022 and Sept. 2022 to perform the analysis, \
    don't assume the analysis is completed before gathering similar information for the subsequent quarters if available."
)

Thought: The user is requesting a comprehensive analysis of Uber's R&D expenditures and revenue in 2022, using information from the quarterly financial reports ending in March 2022, June 2022, and September 2022.
Action: march_2022
Action Input: {'input': 'R&D expenditures and revenue'}
Observation: Research and development expenses increased by $72 million, or 14%, primarily due to a $63 million increase in stock-based compensation expense. Revenue increased by $4.0 billion, or 136%, primarily driven by increases in Mobility Trip volumes as the business recovers from the impacts of COVID-19, and a $1.5 billion increase in Freight revenue resulting primarily from the acquisition of Transplace in the fourth quarter of 2021.
Thought: I have gathered information from the March 2022 quarterly financial report regarding Uber's R&D expenditures and revenue changes.
Action: june_2022
Action Input: {'input': 'R&D expenditures and revenue'}
Observation: Research and development expenses increas

In [32]:
# initiate a step-wise execution
task = agent.create_task(
    "Analyze the changes in Uber R&D expenditures and revenue in year 2022 and provide a comprehensive report, you need to \
    gather information from the quarterly financials ending Mar. 2022, Jun. 2022 and Sept. 2022 to perform the analysis, \
    don't assume the analysis is completed before gathering similar information for the subsequent quarters if available."
)

step_output = agent.run_step(task.task_id)
print(step_output.is_last)

Thought: I need to gather information from the quarterly financials ending March 2022, June 2022, and September 2022 to analyze the changes in Uber's R&D expenditures and revenue in 2022.
Action: march_2022
Action Input: {'input': 'R&D expenditures and revenue in March 2022'}
Observation: Research and development expenses in March 2022 increased by $72 million compared to the same period in 2021. Revenue in March 2022 increased by $4.0 billion compared to the same period in 2021.
False


In [33]:
# let's put human in the loop now
step_output = agent.run_step(
    task.task_id,
    input="Great! Continue with your planned steps, and at last compare the financials data with DiDi for the same period.")

Added user message to memory: Great! Continue with your planned steps, and at last compare the financials data with DiDi for the same period.
Thought: I need to gather information from the quarterly financials ending June 2022 to continue analyzing the changes in Uber's R&D expenditures and revenue in 2022.
Action: june_2022
Action Input: {'input': 'R&D expenditures and revenue in June 2022'}
Observation: R&D expenditures increased by $216 million in June 2022 compared to the same period in 2021, primarily due to increases in stock-based compensation and employee headcount costs. Revenue in June 2022 increased by $4.1 billion, primarily driven by increases in Gross Bookings, with significant contributions from Freight revenue, Mobility Trip volumes, and Delivery revenue.


In [34]:
while not step_output.is_last:
     step_output = agent.run_step(task.task_id)
step_output = agent.finalize_response(task.task_id)

Thought: I need to gather information from the quarterly financials ending September 2022 to complete the analysis of Uber's R&D expenditures and revenue in 2022.
Action: september_2022
Action Input: {'input': 'R&D expenditures and revenue in September 2022'}
Observation: Research and development expenditures in September 2022 increased by $267 million compared to the same period in 2021, primarily due to increases in stock-based compensation and employee headcount costs. Revenue in September 2022 increased by $3.5 billion, primarily driven by increases in Gross Bookings, Mobility Trip volumes, Freight Gross Bookings, and changes in business models in the UK.
Thought: I can answer without using any more tools. I'll use the user's language to provide a comprehensive report on the changes in Uber's R&D expenditures and revenue in 2022.
Answer: In 2022, Uber experienced significant increases in research and development (R&D) expenditures across the quarters. In March 2022, R&D expenses in